<a href="https://colab.research.google.com/github/whs1111/bert-version/blob/master/src/CLASS/Bert-6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
cd gdrive/My Drive/Colab_Notebooks/code


[Errno 2] No such file or directory: 'gdrive/My Drive/Colab_Notebooks/code'
/content/gdrive/My Drive/Colab_Notebooks/code


In [ ]:
#@title
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd ..

In [ ]:
!pip install fast-bert

In [ ]:
from fast_bert.data_cls import BertDataBunch
DATA_PATH = 'bert-version/src/DATA/'   
LABEL_PATH = 'bert-version/src/DATA/'  
# label_cols = ["Low",
#              "Medium",
#              "High",
#              "Critical"]
label_cols = ["GoodsServices",
        "SearchAndRescue",
        "InformationWanted",
        "Volunteer",
        "Donations",
        "MovePeople",
        "FirstPartyObservation",
        "ThirdPartyObservation",
        "Weather",
        "FundRaising",
        "EmergingThreats",
        "NewSubEvent",
        "MultimediaShare",
        "ServiceAvailable",
        "Factoid",
        "Official",
        "News",
        "CleanUp",
        "Hashtags",
        "OriginalEvent",
        "ContextualInformation",
        "Advice",
        "Sentiment",
        "Discussion",
        "Irrelevant"]
             
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='bert-base-uncased',
                          train_file='train.csv',
                          val_file='valid.csv',
                          label_file='labels.csv',
                          text_col='text',
                          label_col=label_cols,
                          batch_size_per_gpu=1,
                          max_seq_length=140,
                          multi_gpu=False,
                          multi_label=True,
                          model_type='bert')
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy
import logging
import torch
logger = logging.getLogger()
device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]

learner = BertLearner.from_pretrained_model(
						databunch,
						pretrained_path='bert-base-uncased',
						metrics=metrics,
						device=device_cuda,
						logger=logger,
						output_dir='bert-version/src/',
						finetuned_wgts_path=None,
						warmup_steps=500,
						multi_gpu=False,
						is_fp16=True,
						multi_label=True,
						logging_steps=50)

In [5]:
torch.cuda.empty_cache()

In [ ]:

learner.fit(epochs=1,
			lr=6e-5,
			validate=True, 	# Evaluate the model after each epoch
			schedule_type="warmup_cosine")

In [7]:
learner.save_model()

In [ ]:
import pandas as pd
from fast_bert.prediction import BertClassificationPredictor
DATA_PATH = '/content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/DATA/'   
LABEL_PATH = '/content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/DATA'  
MODEL_PATH = '/content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/model_out'

predictor = BertClassificationPredictor(
				model_path=MODEL_PATH,
				label_path=LABEL_PATH, # location for labels.csv file
				multi_label=False,
				model_type='bert',
				do_lower_case=True)

file = '/content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/DATA/test.csv'
combi = pd.read_csv(file)
lens = len(combi['text'])
labels = []
texts = []
for i in range(lens):
     label = []
     i = 0
     texts.append(combi['text'][i])
multiple_predictions = predictor.predict_batch(texts)
    #  single_prediction = predictor.predict(combi['text'][i])
#      for j in range(len(single_prediction)):
#        if single_prediction[j][1]>0.4:
#          label.append(single_prediction[j])
#          break
#      labels.append(label)
# numpy_array = np.array(labels)
# np.save('category.npy',numpy_array )

In [ ]:
#@title
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd ..

import torch
import apex
from fastai.text import *
import datetime
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

LOG_PATH=Path('logs/')  
MODEL_PATH=Path('models/') 

if not LOG_PATH.exists():
  LOG_PATH.mkdir()
import logging

args = {
    "run_text": "my_test",
    "max_seq_length": 30,
    "do_lower_case": True,
    "train_batch_size": 16,
    "learning_rate": 6e-5,
    "num_train_epochs": 12.0,
    "warmup_proportion": 0.002,
    "local_rank": -1,
    "gradient_accumulation_steps": 1,
    "fp16": True,
    "loss_scale": 128
}

logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

device = torch.device('cuda')

if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

In [ ]:
!pip install fast-bert==0.1.4

In [ ]:
from fast_bert.data import *
from fast_bert.learner import *
from fast_bert.metrics import *
from pytorch_pretrained_bert.tokenization import BertTokenizer

In [ ]:
DATA_PATH = Path('bert-version/src/DATA/')     
LABEL_PATH = Path('bert-version/src/DATA/')  

BERT_PRETRAINED_MODEL = "bert-base-uncased"

args["do_lower_case"] = True
args["train_batch_size"] = 16
args["learning_rate"] = 6e-5
args["max_seq_length"] = 140
args["fp16"] = True
tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_MODEL, 
                                          do_lower_case=args['do_lower_case'])
label_cols = ["Low",
             "Medium",
             "High",
             "Critical"]
databunch = BertDataBunch(DATA_PATH, LABEL_PATH, tokenizer, train_file='train.csv', val_file='valid.csv',
                          test_data='test.csv', label_file="labels.csv",
                          text_col="text", label_col=label_cols,
                          bs=args['train_batch_size'], maxlen=args['max_seq_length'], 
                          multi_gpu=multi_gpu, multi_label=True)
metrics = [{'name': 'accuracy', 'function': accuracy_multilabel}]
logger.info(args)
learner = BertLearner.from_pretrained_model(databunch, BERT_PRETRAINED_MODEL, metrics, device, logger, 
                                            is_fp16=args['fp16'], loss_scale=args['loss_scale'], 
                                            multi_gpu=multi_gpu,  multi_label=True)
learner.fit(8, lr=args['learning_rate'], schedule_type="warmup_linear")

In [ ]:
learner.save_and_reload('/content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/DATA/', "bert-model")
from fast_bert.prediction import BertClassificationPredictor
import numpy as np 

predictor = BertClassificationPredictor(model_path='bert-version/src/DATA/bert-model.bin', pretrained_path="bert-base-uncased", 
                                        label_path='/content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/DATA/', multi_label=True)

# texts = []
file = '/content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/DATA/test.csv'
combi = pd.read_csv(file)
lens = len(combi['text'])
labels = []
for i in range(lens):
#     texts.append(combi['text'][i])
    label = []
    single_prediction = predictor.predict(combi['text'][i])
    for j in range(len(single_prediction)):
      if single_prediction[j][1]>0.3:
        label.append(single_prediction[0])
    labels.append(label)
numpy_array = np.array(labels)
np.save('category.npy',numpy_array )
label_cols = ["GoodsServices",
        "SearchAndRescue",
        "InformationWanted",
        "Volunteer",
        "FundRaising",
        "Donations",
        "MovePeople",
        "FirstPartyObservation",
        "ThirdPartyObservation",
        "Weather",
        "EmergingThreats",
        "NewSubEvent",
        "MultimediaShare",
        "ServiceAvailable",
        "Factoid",
        "Official",
        "CleanUp",
        "Hashtags",
        "ContextualInformation",
        "News",
        "Advice",
        "Sentiment",
        "Discussion",
        "Irrelevant",
        "OriginalEvent"]
        
# print(single_prediction)

# Batch predictions
# texts = [
#   "this is the first text",
#   "this is the second text"
# ]

# multiple_predictions = predictor.predict(texts)
